In [1]:
import sys
sys.path.append("../")

from compute_embedding import SpeechEmbedding
from speaker_encoder.io import load_config


In [3]:
model_path = "../pretrained_model/best_model.pth.tar"
config_path = "../pretrained_model/config.json"
config = load_config(config_path)

speech_embedding = SpeechEmbedding(config, model_path)

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:None
 | > hop_length:256
 | > win_length:1024


In [4]:
comvoice_path = 

# Compute speaker embeddings
wav_file = "../outputs/audio006.wav"
embedd = speech_embedding.compute_embedding(wav_file)
print(embedd)
print(embedd.shape)


[[ 8.78657922e-02  9.84800085e-02 -2.16337922e-03 -3.96862440e-02
  -2.04127729e-02  5.56823127e-02 -8.83411467e-02  7.41974786e-02
  -4.99743931e-02  1.29765719e-01 -7.85102621e-02  4.23021317e-02
  -8.75026584e-02 -9.32246223e-02 -4.94175553e-02 -2.62239780e-02
  -5.73086180e-02 -2.81425263e-03 -4.57083881e-02  1.58158224e-02
   2.55967323e-02  3.29629555e-02 -6.14315867e-02  3.03938091e-02
  -2.04871688e-02  2.25423034e-02 -4.29750839e-03  6.44263551e-02
   4.72899266e-02 -1.61759909e-02 -2.23914683e-02  9.53922793e-02
  -3.69876102e-02  6.58994988e-02  4.07397710e-02  1.90815311e-02
   8.24525803e-02 -7.24233910e-02 -1.89699773e-02  2.29313094e-02
   1.25566032e-03  6.26777932e-02  2.03067921e-02 -3.70928757e-02
  -1.91252623e-02  6.66978732e-02  2.83426289e-02 -2.06153449e-02
  -2.02723388e-02  9.48714390e-02  2.83090938e-02  7.76994824e-02
  -5.44256680e-02 -7.90656283e-02  1.38044730e-02 -1.10850960e-01
   8.72512907e-03 -1.72353368e-02 -2.37873173e-03  1.17714107e-01
  -9.34036